## Advanced SQL Exercises (with Answers)

Today we will work with the 'chinook' database through SQLite. The chinook database has 11 different tables we can query and is a good database for us to practice our SQL skills. The following diagram shows us how these tables are related to one another in our database.

<img src ='Chinook_Schema.png'>

Run the two following cells to get started working with SQLite on Jupyter. Remember that you need to use the magic functions (%sql or %%sql) when trying to run SQL commands; the former is used for single line commands whilst the latter is for multiple lines.

In [ ]:
!pip install ipython-sql

In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

# General Notes about Syntax


- When using SQL keywords such as SFW(SELECT, FROM, WHERE) it is best practice to use capital letters even though SQL itself is not case sensitive. This produces more human-readable code, and is easier to interpret and pick up where we left off.
- Make sure to end each query with ';'. This is not strictly necessary, but will signify the 'end' of our query
- When setting an alias, you can either explicitly mention 'FirstName' AS first_name or you could just assign the alias after a space 'FirstName' first_name. The method you choose comes down to personal preference
- When using the LIKE operator, the % symbol is a 'wildcard' that could refer to characters of any length. The _ symbol refers to a 'wildcard' single character
- In this toy dataset it is completely fine to select all with * however in real-world we should explicitly choose the columns we want so as to avoid costly computations

# Section 1: Querying Data

## Problem 1

It is tradition in your company to celebrate the birthdays of everyone born in the same month together. As we are approaching the end of April, we need to find out which employees have their birthdays in May. Query the 'employees' table to find this information.

In [2]:
%sql SELECT FirstName, LastName FROM employees WHERE BirthDate LIKE '%-05-%';

 * sqlite:///chinook.db
Done.


FirstName,LastName
Robert,King


## Problem 2

A new law has been passed in parliament that makes it mandatory for companies that have more than or equal to 5 employees in a given city to declare this number to their local city council. Which cities are required to make this report to the city council? Query the 'employees' table for this information.

In [3]:
%%sql SELECT City, COUNT(EmployeeId) AS Number_Of_Employees
FROM employees 
GROUP BY City 
HAVING Number_Of_Employees >= 5;

 * sqlite:///chinook.db
Done.


City,Number_Of_Employees
Calgary,5


## Problem 3

You are performing an audit on all the tracks you have in your database. Can you retrieve the following information:

- The total number of tracks in your database
- The total number of unique albums present in your database
- The number of unique unit prices, and the number of tracks for each of these prices

In [4]:
%sql SELECT COUNT(*) AS 'Total Tracks' FROM tracks;

 * sqlite:///chinook.db
Done.


Total Tracks
3503


In [5]:
%sql SELECT COUNT (DISTINCT AlbumId) AS 'Number of Albums' FROM tracks;

 * sqlite:///chinook.db
Done.


Number of Albums
347


In [6]:
%sql SELECT UnitPrice, COUNT(UnitPrice) AS Amount FROM tracks GROUP BY UnitPrice;

 * sqlite:///chinook.db
Done.


UnitPrice,Amount
0.99,3290
1.99,213


## Problem 4a

A customer has just walked into the store looking to buy an album for his father. He cannot remember exactly which album his dad wanted, but he does remember that this particular album has at least 10 song names with the word 'The' in it. Can you find the AlbumId for this album?

In [7]:
%%sql SELECT AlbumId, COUNT(*) AS Number_of_The 
FROM tracks 
WHERE name LIKE '%The%' 
GROUP BY AlbumId
HAVING Number_of_The > 10;

 * sqlite:///chinook.db
Done.


AlbumId,Number_of_The
253,16


## Problem 4b

Now that you found the AlbumId, look up the tables to find the name of the artist, and the name of the album 

In [8]:
%sql SELECT * FROM albums LIMIT 1;

 * sqlite:///chinook.db
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1


In [9]:
%sql SELECT * FROM artists LIMIT 1;

 * sqlite:///chinook.db
Done.


ArtistId,Name
1,AC/DC


In [10]:
%%sql SELECT Title AS Album, Name AS Artist
FROM albums al 
INNER JOIN artists a 
ON a.ArtistId = al.ArtistId
WHERE AlbumId = 253;

 * sqlite:///chinook.db
Done.


Album,Artist
"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic)


## Problem 4c

Query the data to find the artist name and albums for all records with albumId between 100 and 110

In [ ]:
%%sql SELECT AlbumId, Title AS Album, Name AS Artist_Name
FROM albums al 
INNER JOIN artists a 
ON a.ArtistId = al.ArtistId
WHERE AlbumId BETWEEN
100 AND 110;

## Problem 5

Working with the tracks and invoice_items tables, can you determine which genres are generating the most revenue for your business? This would require the following information

- Quantity of tracks sold by GenreId
- Revenue generated for each GenreId IF the revenue is greater than 100

Hint: You need to use a JOIN and a GROUP BY

Hint 2: Revenue is given by Price * Quantity

In [11]:
%%sql SELECT t.GenreId, 
COUNT(i.Quantity) AS Quantity, 
ROUND(SUM(i.UnitPrice * i.Quantity),2) AS Revenue

FROM invoice_items i

INNER JOIN tracks t
ON t.TrackId = i.TrackId

GROUP BY GenreId
HAVING Revenue > 100;

 * sqlite:///chinook.db
Done.


GenreId,Quantity,Revenue
1,835,826.65
3,264,261.36
4,244,241.56
7,386,382.14


In [12]:
%sql SELECT * FROM genres LIMIT 5;

 * sqlite:///chinook.db
Done.


GenreId,Name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll


## Problem 6

What are the names of the first five songs that will play from the 'Classical' playlist?

Hint: Query the playlist_track and tracks tables for this information

In [13]:
%%sql SELECT name FROM playlist_track pt
INNER JOIN tracks t
ON pt.TrackId = t.TrackId
WHERE PlaylistId == (SELECT PlaylistId FROM playlists WHERE Name == 'Classical')
LIMIT 5;

 * sqlite:///chinook.db
Done.


Name
Intoitus: Adorate Deum
"Miserere mei, Deus"
Canon and Gigue in D Major: I. Canon
"Concerto No. 1 in E Major, RV 269 ""Spring"": I. Allegro"
"Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace"


## Problem 7

Your company is running a promotion to provide a huge discount coupon to three customers that spent the most money in a single transaction in 2013.  Can you determine the CustomerId for these customers and the amount they paid? 

Hint: Use the invoices table

In [14]:
%%sql SELECT CustomerId, Total 
FROM invoices 
WHERE InvoiceDate 
LIKE '2013-%' 
ORDER BY Total DESC LIMIT 3;

 * sqlite:///chinook.db
Done.


CustomerId,Total
6,25.86
39,13.86
18,13.86


## Problem 8

Querying the same table, can you determine the top 5 countries from which you are deriving most of your sales? You can infer this from the 'BillingCountry' column.

Hint: Provide your answer in descending order

In [15]:
%%sql SELECT BillingCountry, ROUND(SUM(Total)) AS Total
FROM invoices 
GROUP BY BillingCountry 
ORDER BY 
ROUND(SUM(Total)) DESC 
LIMIT 5;

 * sqlite:///chinook.db
Done.


BillingCountry,Total
USA,523.0
Canada,304.0
France,195.0
Brazil,190.0
Germany,156.0


## Problem 9

One of your customers called customer service earlier today to enquire about a free gift that he/she never received. You wrote down his/her contact information, but spilled coffee all over the post-it note! 

The only legible words on your note are 'gmail.com' and a first name that seems to begin with 'He'. You also remember the customer has a thick European accent. 

Querying the Customers table, can you find out who this and his/her address to post that free gift?

In [16]:
%%sql SELECT * FROM Customers 
WHERE Email LIKE '%@gmail.com' 
AND FirstName LIKE '%He%'

 * sqlite:///chinook.db
Done.


CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4


## Problem 10

You're about to go on an hour long road trip, and you want to select a song that lasts longer than one hour. Which tracks fulfil this condition?

In [17]:
%%sql SELECT Name, (Milliseconds/3600002) AS Hour 
FROM tracks;

 * sqlite:///chinook.db
Done.


Name,Hour
For Those About To Rock (We Salute You),0
Balls to the Wall,0
Fast As a Shark,0
Restless and Wild,0
Princess of the Dawn,0
Put The Finger On You,0
Let's Get It Up,0
Inject The Venom,0
Snowballed,0
Evil Walks,0


In [ ]:
%%sql SELECT Name, TrackId, ROUND((Milliseconds*0.001)/60, 2) AS Duration 
FROM tracks 
WHERE Duration > 60
ORDER BY Duration DESC;

## Problem 11

What is the average duration for the all songs in your database? Give your answer in minutes rounded to two decimal places.

In [18]:
%%sql SELECT ROUND(AVG((Milliseconds*0.001)/60), 2) AS Duration
FROM tracks;

 * sqlite:///chinook.db
Done.


Duration
6.56


## Problem 12

Which artist has produced the most albums?

Hint: This can be acomplished with either a SubQuery or with a Join!

In [19]:
%%sql SELECT Name
FROM artists 
WHERE ArtistId = 
(SELECT ArtistId 
 FROM albums 
 GROUP BY ArtistId 
 ORDER BY COUNT(*) 
 DESC LIMIT 1);

 * sqlite:///chinook.db
Done.


Name
Iron Maiden


## Problem 13

The new CEO of our company has an obsession with streamlining data storage in the company. He wants to find tracks with the lowest Revenue per byte ratio in an attempt to improve our cost efficiency. Find the 10 worst performing songs in terms of Revenue / Byte. For easier interpretation please include a column 'Mb' that displays the number of bytes in megabytes as the CEO is only familiar with this unit.

Hint: Use invoice_items and tracks tables

In [20]:
%%sql SELECT t.Name, t.trackid, sum(Quantity)*i.unitprice AS Revenue, 
Bytes/1000000 AS Mb, (sum(Quantity)*i.unitprice)/Bytes AS Dollarspb
FROM tracks t, invoice_items i
WHERE t.TrackId = i.TrackId
GROUP BY t.TrackId
ORDER BY Dollarspb
LIMIT 5;

 * sqlite:///chinook.db
Done.


Name,TrackId,Revenue,Mb,Dollarspb
Through a Looking Glass,3224,1.99,1059,1.878162663119135e-09
Occupation / Precipice,2820,1.99,1054,1.887286425492465e-09
Dave,2910,1.99,574,3.464932098674601e-09
The Magnificent Warriors,3235,1.99,570,3.4902959039893752e-09
The Lost Warrior,3231,1.99,558,3.5607425733601093e-09
